In [0]:
from keras.layers import Dense, Dropout,  Conv2D, Input, Lambda, Flatten, TimeDistributed
from keras.layers import Add, Reshape, MaxPooling2D, Concatenate, Embedding, RepeatVector, Multiply
from keras.models import Model
from keras import backend as K

import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.engine.topology import Layer
import tensorflow as tf

In [0]:
def main_attention(x, l=6*6, d=64*3 , dv=8*3, nv = 8):
   
    v = Dense(d, activation = 'relu')(x)
    k = Dense(d, activation = 'relu')(x)
    q = Dense(d, activation = 'relu')(x)
   
    v = Reshape([l,dv,nv])(v)  
    k = Reshape([l,dv,nv])(k)  
    q = Reshape([l,dv,nv])(q)
   
    new = Multiply()([q,k])
    new = Activation('softmax')(new)
   
    new1 = Multiply()([new,v])
    new1 = Reshape([l,d])(new1)
    out = Add()([new1,x])
   
    return out

In [0]:
def multi_head(l=6*6, d=64*3 , dv=8*3, nv = 8):
   
    input_img = Input(shape = (l,d))
   
    out_1 = main_attention(input_img, l=6*6, d=64*3 , dv=8*3, nv = 8)
    out_2 = main_attention(input_img, l=6*6, d=64*3 , dv=8*3, nv = 8)
    out = Concatenate(axis = 2)([out_1,out_2])
    att_model = Model(input = input_img, output = out)
    print(att_model.summary())
    return att_model


In [0]:
'''
def MultiHeadsAttModel(l=8*8, d=512, dv=64, dout=512, nv = 8 ):
    v1 = Input(shape = (l, d))
    q1 = Input(shape = (l, d))
    k1 = Input(shape = (l, d))

    v2 = Dense(dv*nv, activation = "relu")(v1)
    q2 = Dense(dv*nv, activation = "relu")(q1)
    k2 = Dense(dv*nv, activation = "relu")(k1)

    v = Reshape([l, nv, dv])(v2)
    q = Reshape([l, nv, dv])(q2)
    k = Reshape([l, nv, dv])(k2)
        
    att = Lambda(lambda x: K.batch_dot(x[0],x[1] ,axes=[-1,-1]) / np.sqrt(dv),
                 output_shape=(l, nv, nv))([q,k])# l, nv, nv
    att = Lambda(lambda x:  K.softmax(x) , output_shape=(l, nv, nv))(att)

    out = Lambda(lambda x: K.batch_dot(x[0], x[1],axes=[1,1]),  output_shape=(l, nv, dv))([att, v])
    out = Reshape([l, d])(out)
    
    out = Add()([out, q1])

    out = Dense(dout, activation = "relu")(out)

    return  Model(inputs=[q1,k1,v1], outputs=out)

class NormL(Layer):

    def __init__(self, **kwargs):
        super(NormL, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.a = self.add_weight(name='kernel', 
                                      shape=(1,input_shape[-1]),
                                      initializer='ones',
                                      trainable=True)
        self.b = self.add_weight(name='kernel', 
                                      shape=(1,input_shape[-1]),
                                      initializer='zeros',
                                      trainable=True)
        super(NormL, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        eps = 0.000001
        mu = K.mean(x, keepdims=True, axis=-1)
        sigma = K.std(x, keepdims=True, axis=-1)
        ln_out = (x - mu) / (sigma + eps)
        return ln_out*self.a + self.b

    def compute_output_shape(self, input_shape):
        return input_shape

'''

In [0]:
nb_classes = 10

In [0]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)


X_train original shape (60000, 28, 28)
y_train original shape (60000,)


In [0]:
X_train = X_train.reshape(60000, 28,28,1)
X_test = X_test.reshape(10000, 28,28,1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

Training matrix shape (60000, 28, 28, 1)
Testing matrix shape (10000, 28, 28, 1)


In [0]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [0]:
# inp = Input(shape = (28,28,1))
# x = Conv2D(32,(2,2),activation='relu', padding='same')(inp)
# x = MaxPooling2D(pool_size=(2, 2))(x)
# x = Conv2D(64,(2,2),activation='relu')(x)
# x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
# x = Conv2D(64*3,(2,2),activation='relu')(x)

# x = Reshape([6*6,64*3])(x)    
# att = MultiHeadsAttModel(l=6*6, d=64*3 , dv=8*3, dout=32, nv = 8 )
# x = att([x,x,x])
# x = Reshape([6,6,32])(x)   
# x = NormL()(x)
# x = Flatten()(x) 
# x = Dense(256, activation='relu')(x)
# x = Dense(10, activation='softmax')(x)
inp = Input((28, 28, 1), name='input_layer')
x = Conv2D(32,(2,2),activation='relu', padding='same')(inp)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64,(2,2),activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = Conv2D(64*3,(2,2),activation='relu')(x)
x = Reshape([6*6,64*3])(x)  
att = multi_head(l=6*6, d=64*3 , dv=8*3, nv = 8)
out = att(x)
out = Dense(32, activation = 'relu')(out)
#x = Reshape([6,6,32])(x)  
x = Flatten()(out)
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

model = Model(input = inp, output = x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  app.launch_new_instance()


In [0]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 32)        160       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 64)        8256      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 192)         49344     
_________________________________________________________________
reshape_5 (Reshape)          (None, 36, 192)           0   

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [0]:
model.fit(X_train, Y_train,
              batch_size=128, 
              epochs=100,
              verbose=1,          
              validation_data=(X_test, Y_test)
             )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/100





60000/60000 [==============================] - 30s 496us/step - loss: 0.1994 - acc: 0.9384 - val_loss: 0.0766 - val_acc: 0.9756
Epoch 2/100
60000/60000 [==============================] - 15s 257us/step - loss: 0.0503 - acc: 0.9841 - val_loss: 0.0358 - val_acc: 0.9873
Epoch 3/100
60000/60000 [==============================] - 15s 255us/step - loss: 0.0337 - acc: 0.9894 - val_loss: 0.0314 - val_acc: 0.9905
Epoch 4/100
60000/60000 [==============================] - 15s 254us/step - loss: 0.0258 - acc: 0.9922 - val_loss: 0.0304 - val_acc: 0.9901
Epoch 5/100
60000/60000 [==============================] - 15s 253us/step - loss: 0.0202 - acc: 0.9935 - val_loss: 0.0329 - val_acc: 0.9897
Epoch 6/100
60000/60000 [==============================] - 15s 252us/step - loss: 0.0173 - acc: 0.9943 - val_loss: 0.0333 - val_acc: 0.9898
Epoch 7/